# Sistema de recomendacion

## Importación de librerias

In [1]:
 # Importamos las librerias correspondientes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importación de datasets

In [2]:
# Leemos los dataset exportados con anterioridad
games= pd.read_parquet('Dataset_ETL/steam_games.parquet',engine='pyarrow')
reviews=pd.read_parquet('Dataset_ETL/user_reviews.parquet',engine='pyarrow')
items=pd.read_parquet('Dataset_ETL/users_items.parquet',engine='pyarrow')

In [3]:
games.head()

,publisher,genres,app_name,release_date,price,id,developer,average price
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,3.473871
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,25.29,643980,Secret Level SRL,25.290000
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,6.607647
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd,7.823333
5,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,2018-01-04,9.99,774276,Poppermost Productions,26.790000


In [5]:
reviews.head()

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,1
3,js41637,251610,True,2
4,js41637,227300,True,1


In [4]:
recommends = pd.merge(reviews,games,left_on='item_id',right_on='id')

In [6]:
recommends=recommends.iloc[:,[0,1,3,6]]

In [54]:
recommends = recommends[recommends.sentiment_analysis == 1]

In [10]:
recommends

,user_id,item_id,sentiment_analysis,app_name
0,76561197970982479,1250,1,Killing Floor
1,76561197970982479,22200,1,Zeno Clash
2,js41637,227300,0,Euro Truck Simulator 2
3,js41637,239030,0,"Papers, Please"
4,evcentric,248820,1,Risk of Rain
...,...,...,...,...
49075,76561198312638244,70,1,Half-Life
49076,76561198312638244,362890,1,Black Mesa
49077,LydiaMorley,273110,0,Counter-Strike Nexon: Zombies
49078,LydiaMorley,730,1,Counter-Strike: Global Offensive


In [8]:
recommends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49080 entries, 0 to 49079
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49080 non-null  object
 1   item_id             49080 non-null  object
 2   sentiment_analysis  49080 non-null  int64 
 3   app_name            49080 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [9]:
# Convertir la escala 2, 1, 0 a 1, 0, -1
recommends.sentiment_analysis = recommends.sentiment_analysis.map({2:1,1:0,0:-1})

C:\Users\JAY\AppData\Local\Temp\ipykernel_2324\3243841837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommends.sentiment_analysis = recommends.sentiment_analysis.map({2:1,1:0,0:-1})


In [11]:
# Concatenar los valores duplicados en una cadena
df_pivot = recommends.pivot_table(index='user_id', columns='app_name', values='sentiment_analysis', aggfunc='mean')

In [12]:
df_pivot.fillna(0,inplace=True)

In [13]:
df_pivot

app_name,! That Bastard Is Trying To Steal Our Gold !,//N.P.P.D. RUSH//- The milk of Ultraviolet,0RBITALIS,"10,000,000",100% Orange Juice,100% Orange Juice - Krila & Kae Character Pack,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,...,liteCam Game: 100 FPS Game Capture,nail'd,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter Classic,theHunter: Primal,Астролорды: Облако Оорта
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ace--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ionex--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2SV-vuLB-Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Azsael-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zvanik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zwanzigdrei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zy0705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
print(df_pivot.iloc[9,:].value_counts())

-PRoSlayeR-
 0.0    2921
 1.0       1
-1.0       1
Name: count, dtype: int64


In [52]:
print(df_pivot.iloc[0,:].dtype)

object


In [ ]:
df_pivot.iloc[:,0].astype('int')

In [21]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23005 entries, --000-- to zyr0n1c
Columns: 2923 entries, ! That Bastard Is Trying To Steal Our Gold ! to Астролорды: Облако Оорта
dtypes: float64(2923)
memory usage: 513.2+ MB


He podido realizar la tabla con los datos de los usuarios y con la parte de las calificaciones
Toca realizar en el dataset recommends, la similitud del coseno, esto hara que los usuarios tengan recomendaciones de juegos similares con una lista de 5 juegos similares que puedan gustarle.

## Similitud del coseno

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [22]:
# Asumamos que 'df_pivot' es tu tabla de usuarios y juegos con valores de sentiment_analysis
# Asegúrate de que 'user_id' sea el índice del DataFrame
# df_pivot = df_pivot.set_index('user_id')

# Calculamos la similitud del coseno entre todos los usuarios
user_similarity = cosine_similarity(df_pivot)

# Convertimos la matriz de similitud en un DataFrame para que sea más fácil manejarlo
user_similarity_df = pd.DataFrame(user_similarity, index=df_pivot.index, columns=df_pivot.index)

# Ver la matriz de similitud
print(user_similarity_df)

user_id       --000--   --ace--  --ionex--  -2SV-vuLB-Kg  -Azsael-  -Beave-  \
user_id                                                                       
--000--           0.0  0.000000        0.0      0.000000       0.0      0.0   
--ace--           0.0  1.000000        0.0      0.408248       0.0      0.0   
--ionex--         0.0  0.000000        0.0      0.000000       0.0      0.0   
-2SV-vuLB-Kg      0.0  0.408248        0.0      1.000000       0.0      0.0   
-Azsael-          0.0  0.000000        0.0      0.000000       1.0      0.0   
...               ...       ...        ...           ...       ...      ...   
zvanik            0.0  0.000000        0.0      0.000000       0.0      0.0   
zwanzigdrei       0.0  0.707107        0.0      0.577350       0.0      0.0   
zy0705            0.0  0.707107        0.0      0.577350       0.0      0.0   
zynxgameth        0.0  0.000000        0.0      0.000000       0.0      0.0   
zyr0n1c           0.0 -0.316228        0.0     -0.25

In [27]:
# Elegimos el usuario para el que queremos hacer la recomendación
target_user = '--000--'  # Ejemplo con el usuario 1

# Ordenamos las similitudes del usuario en orden descendente
similar_users = user_similarity_df.loc[target_user].sort_values(ascending=False)

# Excluimos al propio usuario (ya que la similitud consigo mismo es 1)
similar_users = similar_users.drop(target_user)

# Vemos los usuarios más similares
print(similar_users.head())

user_id
zyr0n1c         0.0
--ace--         0.0
--ionex--       0.0
-2SV-vuLB-Kg    0.0
-Azsael-        0.0
Name: --000--, dtype: float64


In [29]:
# Obtenemos los juegos que el usuario objetivo ya ha calificado
user_games = df_pivot.loc[target_user]

# Filtramos los juegos que el usuario ya ha jugado o que no le gustaron (valores 0 o negativos)
games_user_played = user_games[user_games > 0].index

# Ahora buscamos los juegos que han disfrutado los usuarios más similares
# Seleccionamos los 5 usuarios más similares
top_similar_users = similar_users.head(5).index

# Filtramos los juegos que estos usuarios disfrutaron (con puntuaciones positivas)
recommended_games = df_pivot.loc[top_similar_users].mean().sort_values(ascending=False)

# Excluimos los juegos que ya jugó el usuario
recommended_games = recommended_games.drop(games_user_played)

# Recomendamos los 5 juegos mejor valorados entre usuarios similares
top_5_recommendations = recommended_games.head(5)

print("Juegos recomendados para el usuario:", list(top_5_recommendations.index))

Juegos recomendados para el usuario: ['Counter-Strike: Global Offensive', 'Loadout', "Garry's Mod", 'Team Fortress 2', 'The Binding of Isaac']


Exportamos los datasets "user_similarity_df" que es el dataset que esta calculada la similitud del coseno en los usuarios.

Y exportamos el dataset "df_pivot" donde esta la tabla pivote que hemos obtenido para el analisis de las recomendacion de los usuarios.

In [30]:
df_pivot.to_parquet('Dataset_funciones/df_recomendacion_PivotTable.parquet',engine = 'pyarrow')
user_similarity_df.to_parquet('Dataset_funciones/df_recomendacion_UserSimilarity.parquet',engine = 'pyarrow')

In [31]:
df_UserSimilarity=pd.read_parquet('./Dataset_funciones/df_recomendacion_UserSimilarity.parquet',engine='pyarrow')

In [32]:
df_UserSimilarity

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-I_AM_EPIC-,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zunbae,zuzuga2003,zv_odd,zvanik,zwanzigdrei,zy0705,zynxgameth,zyr0n1c
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000
--ace--,0.0,1.000000,0.0,0.408248,0.0,0.0,0.707107,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.707107,0.707107,0.0,-0.316228
--ionex--,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000
-2SV-vuLB-Kg,0.0,0.408248,0.0,1.000000,0.0,0.0,0.577350,0.0,0.0,0.0,...,0.0,0.0,0.0,0.57735,0.0,0.0,0.577350,0.577350,0.0,-0.258199
-Azsael-,0.0,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zvanik,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,1.0,0.000000,0.000000,0.0,0.000000
zwanzigdrei,0.0,0.707107,0.0,0.577350,0.0,0.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,1.000000,1.000000,0.0,-0.447214
zy0705,0.0,0.707107,0.0,0.577350,0.0,0.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,1.000000,1.000000,0.0,-0.447214
